# Generative Panning with Outpainting

* **Kayla Akyüz** - [**GitHub**](https://github.com/kaylaa0)

In [ ]:
import cv2
import numpy as np
import requests
import json
from PIL import Image
from io import BytesIO

host = "http://127.0.0.1:8888" # Adress of the Fooocus API
model = "juggernautXL_juggernautX.safetensors" # Model for generation

In [ ]:
def warp_image(image, angle_degrees, direction):
    # Convert angle from degrees to radians
    angle_radians = np.radiaans(angle_degrees)

    # Get image dimensions
    height, width = image.shape[:2]

    offset = 0
    mask_padding = 15
    mask_destionation = np.float32([
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0],
            [0, 0]
        ])

    if direction == 'up' or direction == 'down':
        offset = int(width * np.tan(angle_radians) * 0.5)
    elif direction == 'left' or direction == 'right':
        offset = int(height * np.tan(angle_radians) * 0.5)
        
    height = height + (offset*2)
    width = width + (offset*2)
    image = cv2.copyMakeBorder(image, offset, offset, offset, offset, cv2.BORDER_CONSTANT, value=[0, 0, 0])
    
    # Define 8 points for perspective transformation
    src_points = np.float32([
        [offset, offset],                 # Top-left corner
        [(width - 1) // 2, offset],        # Top-middle
        [width - 1 - offset, offset],         # Top-right corner
        [width - 1 - offset, height // 2],# Right-middle
        [width - 1 - offset, height - 1 - offset], # Bottom-right corner
        [(width - 1) // 2, height - 1 - offset],# Bottom-middle
        [offset, height - 1  - offset],        # Bottom-left corner
        [offset, height // 2]        # Left-middle
    ])

    if direction == 'up':
        dst_points = np.float32([
            [offset*2, offset*2],
            [(width - 1 )//2, offset*2],
            [(width - 1 ) - (offset*2), offset*2],
            [width - 1 - offset, (height -1)//2],
            [width - 1, (height -1)],
            [(width - 1 )//2, (height -1)],
            [0, height - 1],
            [offset, (height -1)//2]
        ])
        mask_destionation = np.float32([
            [offset*2 + mask_padding, offset*2 + mask_padding],
            [(width - 1) - (offset*2) - mask_padding, offset*2 + mask_padding],
            [(width - 1) - (mask_padding*2), (height -1)],
            [(mask_padding*2), (height -1)]
        ])
            
    elif direction == 'down':
        dst_points = np.float32([
            [0, 0],
            [(width - 1 )//2, 0],
            [width - 1, 0],
            [width - 1 - offset, (height -1)//2],
            [width - 1 - (offset*2), (height - 1) - (offset*2)],
            [(width - 1 )//2, (height - 1) - (offset*2)],
            [offset*2, (height - 1) - (offset*2)],
            [offset, (height -1)//2]
        ])
        mask_destionation = np.float32([
            [mask_padding*2, 0],
            [width - 1 - (mask_padding*2), 0],
            [width - 1 - (offset*2) - (mask_padding), (height - 1) - (offset*2) - (mask_padding)],
            [(offset*2) + mask_padding, (height - 1) - (offset*2) - mask_padding]
        ])
       
    elif direction == 'left':
        dst_points = np.float32([
            [offset*2, offset*2],
            [(width - 1 )//2, offset],
            [width - 1 , 0],
            [width - 1 , (height -1)//2],
            [width - 1 , height - 1],
            [(width - 1 )//2, (height -1) - (offset)],
            [offset*2, (height -1)-(offset*2)],
            [offset*2, (height -1)//2]
        ])
        mask_destionation = np.float32([
            [offset*2 + mask_padding, offset*2 + mask_padding],
            [width - 1, mask_padding*2],
            [width - 1, (height -1) - (mask_padding*2)],
            [offset*2 + mask_padding, (height -1)- (offset*2) - mask_padding]
        ])
    elif direction == 'right':
        dst_points = np.float32([
            [0, 0],
            [(width - 1 )//2, offset],
            [(width - 1) - (offset*2) , offset*2],
            [(width - 1) - (offset*2) , (height -1)//2],
            [(width - 1) - (offset*2) , (height - 1) - offset*2],
            [(width - 1 )//2, (height -1) - offset],
            [0, height - 1],
            [0, (height -1)//2]
        ])
        mask_destionation = np.float32([
            [0, mask_padding*2],
            [(width - 1) - (offset*2) - mask_padding, offset*2 + mask_padding],
            [(width - 1) - (offset*2) - mask_padding, (height - 1) - (offset*2) - mask_padding],
            [0, (height -1) - (mask_padding*2)],
        ])

       
    else:
        raise ValueError("Invalid direction. Use 'left', 'right', 'up', or 'down'.")

    # Calculate the perspective transformation matrix
    matrix, _ = cv2.findHomography(src_points, dst_points)

    # Apply the perspective transformation
    warped_image = cv2.warpPerspective(image, matrix, (width, height), flags=cv2.INTER_CUBIC)

    # Create a mask image
    mask_image = np.zeros_like(warped_image)
    mask_image = cv2.fillPoly(mask_image, [np.int32([[0, 0], [warped_image.shape[1], 0], [warped_image.shape[1], warped_image.shape[0]], [0, warped_image.shape[0]]])], (255, 255, 255))

    # Calculate the mask perspective transformation matrix
    mask_image = cv2.fillPoly(mask_image, [np.int32(mask_destionation)], (0, 0, 0))

    return warped_image, mask_image, offset


In [ ]:
def convert_image_to_byte(image):
  image_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
  bytes_io = BytesIO()
  image_pil.save(bytes_io, format='PNG')
  return bytes_io.getvalue()

In [ ]:
def inpaint_outpaint(params: dict, input_image: bytes, input_mask: bytes = None) -> dict:
    """
    example for inpaint outpaint v1
    """
    response = requests.post(url=f"{host}/v1/generation/image-inpaint-outpaint",
                        data=params,
                        files={"input_image": input_image,
                               "input_mask": input_mask})
    return response.json()

In [ ]:
def run_warp_pipeline(image, degree, direction, save=False, display=False):
    warped_image, mask_image, offset = warp_image(image, degree, direction)
    if save:
        cv2.imwrite('warped_image.png', warped_image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
        cv2.imwrite('mask_image.png', mask_image, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    if display:
        cv2.imshow('Warped Image', warped_image)
        cv2.imshow('Mask Image', mask_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        
    return convert_image_to_byte(warped_image), convert_image_to_byte(mask_image), offset

In [ ]:
def run_outpaint_pipeline(source, mask, prompt="", async_process=True):
  result = inpaint_outpaint(params={
                            "prompt": prompt,
                            "negative_prompt": "dark, shadow, dark shadows, columns, obstructions, blocked view, close object, close-up, frame, obstacles, nearby objects, rear-view mirror, looking out car, looking out of a car, logo, banner, UI, HUD, GUI",
                            "async_process": async_process,
                            "base_model_name": model
                            },
                          input_image=source,
                          input_mask=mask)
  # 
  return result

In [ ]:
# Load your image
image = cv2.imread('game.png')

angle = 5
direction = 'left'

# Run the warp pipeline
warped_image, mask_image, _ = run_warp_pipeline(image, angle, direction, save=True, display=False)

# Run the outpaint pipeline
result = run_outpaint_pipeline(warped_image, mask_image)

print(json.dumps(result, indent=4, ensure_ascii=False))


In [ ]:
def continuous_pipeline(image, direction, degree, times, prompt):
    for _ in range(times):
        warped_image, mask_image, offset = run_warp_pipeline(image, degree, direction, save=False)
        result = run_outpaint_pipeline(warped_image, mask_image, prompt=prompt, async_process=False)
        # Fetch the image from the URL and read it into OpenCV format
        response = requests.get(result[0]['url'])
        image_data = BytesIO(response.content)
        image_array = np.asarray(bytearray(image_data.read()), dtype=np.uint8)
        image = cv2.imdecode(image_array, cv2.IMREAD_COLOR)

        # crop from the opposite direction as much as the offset
        if direction == 'up':
            image = image[:-offset*2, offset:-offset, :]
        elif direction == 'down':
            image = image[offset*2:, offset:-offset, :]
        elif direction == 'left':
            image = image[offset:-offset, :-offset*2, :]
        elif direction == 'right':
            image = image[offset:-offset, offset*2:, :]
    return image

In [ ]:
def look_around(image, degree_increment, path, save = False, step_increment = 1, prompt = ""):
  img = image
  image_no = 0
  for direction in path:
    img = continuous_pipeline(img, direction, degree_increment, step_increment, prompt)
    if save:
      cv2.imwrite(str(image_no) + '_' + direction + '.png', img, [cv2.IMWRITE_PNG_COMPRESSION, 0])
    image_no += 1
    


In [ ]:
path_image = cv2.imread('game.png')

path = ['right'] * 35
path_angle = 5
look_around(path_image, path_angle, path)